In [ ]:
import os
import pydot

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from IPython.display import HTML, SVG, display
from pydrake.visualization import ModelVisualizer
from pydrake.all import Simulator, StartMeshcat, DiagramBuilder, MeshcatVisualizer, namedview

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.scenarios import UnitInertia

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
def make_stack():
    output = ''
    num_layers = 18 # actual number is 18, but can use a smaller number to speed up sim setup
    num_blocks_per_layer = 3
    block_width = 0.05
    block_height = 0.03
    table_height = 0.03
    inertia = UnitInertia.SolidBox(block_width, block_)
    for i in range(num_layers):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(num_blocks_per_layer):
            x = (block_width * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (block_width * (j - 1))
            z = block_height * (i + 0.5) + table_height
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output

In [12]:
scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: table_top_link
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0.2, 0.3, 0.015]
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""

driver_data = """
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""

scenario_data += make_stack()
scenario_data += driver_data
scenario = load_scenario(data = scenario_data)
builder = DiagramBuilder()

station = builder.AddSystem(MakeHardwareStation(scenario, package_xmls=['./package.xml']))
plant = station.GetSubsystemByName("plant")
controller_plant = station.GetSubsystemByName("iiwa.controller").get_multibody_plant_for_control()

visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)

diagram = builder.Build()

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyMutableContextFromRoot(context)
plant_context = plant.GetMyMutableContextFromRoot(context)

q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
station.GetInputPort("iiwa.position").FixValue(station_context, q0)

station.GetInputPort("wsg.position").FixValue(station_context, [0])

simulator.AdvanceTo(0.1)

INFO:drake:Meshcat listening for connections at http://localhost:7009
==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [ ]:
# pick central target
# push block forward or back
# grab firmly and pull out
# move without knocking over stack
# place on top of stack

In [ ]:
def push_targets(block):
    layer, index = block
    layer_in_x_direction = layer % 2 == 0
    if layer_in_x_direction:
        target_x = 
        target_y = block_length
    else:
        target_x = block_width * (j - 1)
        target_y = block_width * (j - 1)
    target_z = block_height * (i + 0.5) + table_height
    return np.array([target_x, target_y, target_z])